In [4]:
import FinanceDataReader as fdr
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
import pickle, time, os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# 모든 컬럼 출력
pd.set_option('display.max_columns', None)
# 모든 행 출력
pd.set_option('display.max_rows', None)
# 열 너비 무제한(문자열 길어도 줄바꿈 없이 출력)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
# 영문 컬럼명 -> 한국어 해석
# 재무제표 영문 → 한국어 매핑 (손익/자산/부채/자본 통합)
# 재무제표 영문 → 한국어 매핑 (손익/자산/부채/자본 통합)
FIN_COL_MAP_KR_BS = {
    # 주식/자본
    "Treasury Shares Number": "자사주 수",
    "Ordinary Shares Number": "보통주 수",
    "Share Issued": "발행주식 수",
    "Common Stock": "보통주 자본금",
    "Preferred Stock": "우선주 자본금",
    "Additional Paid In Capital": "주식발행초과금",
    "Retained Earnings": "이익잉여금",
    "Treasury Stock": "자사주 금액",
    "Stockholders Equity": "총자본/주주지분(비지배제외)",
    "Total Equity Gross Minority Interest": "총자본/주주지분(비지배포함)",

    # 부채
    "Total Liabilities Net Minority Interest": "총부채(비지배제외)",
    "Total Non Current Liabilities Net Minority Interest": "비유동부채(비지배제외)",
    "Long Term Debt": "장기차입금",
    "Current Debt": "단기차입금",
    "Current Liabilities": "유동부채",
    "Accounts Payable": "매입채무",

    # 자산
    "Total Assets": "총자산",
    "Total Non Current Assets": "비유동자산",
    "Current Assets": "유동자산",
    "Cash And Cash Equivalents": "현금 및 현금성자산",
    "Inventory": "재고자산",
    "Accounts Receivable": "매출채권",
    "Net PPE": "순유형자산",
    "Goodwill": "영업권",

    # 자본 관련 조정
    "Tangible Book Value": "유형자산 기반 순자산",
    "Invested Capital": "투자자본",
    "Total Capitalization": "총자본(부채+자본)",
    "Other Equity Adjustments": "기타 자본조정",
}


FIN_COL_MAP_KR_YF = {
    "Tax Effect Of Unusual Items": "비정상 항목 세금영향",
    "Tax Rate For Calcs": "계산용 세율",
    "Normalized EBITDA": "정상화 EBITDA",
    "Total Unusual Items": "총 비정상 항목",
    "Total Unusual Items Excluding Goodwill": "총 비정상 항목(영업권 제외)",
    "Net Income From Continuing Operation Net Minority Interest": "계속영업 순이익(비지배지분 제외)",
    "Reconciled Depreciation": "조정 감가상각비",
    "Reconciled Cost Of Revenue": "조정 매출원가",
    "EBITDA": "EBITDA",
    "EBIT": "영업이익(EBIT)",
    "Net Interest Income": "순이자수익",
    "Interest Expense": "이자비용",
    "Interest Income": "이자수익",
    "Normalized Income": "정상화 순이익",
    "Net Income From Continuing And Discontinued Operation": "계속·중단영업 순이익",
    "Total Expenses": "총비용",
    "Total Operating Income As Reported": "보고된 총영업이익",
    "Diluted Average Shares": "희석평균주식수",
    "Basic Average Shares": "기본평균주식수",
    "Diluted EPS": "희석주당순이익",
    "Basic EPS": "기본주당순이익",
    "Diluted NI Availto Com Stockholders": "희석 순이익(보통주주 귀속)",
    "Net Income Common Stockholders": "보통주주 순이익",
    "Net Income": "순이익",
    "Net Income Including Noncontrolling Interests": "순이익(비지배지분 포함)",
    "Net Income Continuous Operations": "계속영업 순이익",
    "Tax Provision": "법인세 비용",
    "Pretax Income": "세전이익",
    "Other Income Expense": "기타수익/비용",
    "Other Non Operating Income Expenses": "영업외 기타수익/비용",
    "Special Income Charges": "특별손익",
    "Restructuring And Mergern Acquisition": "구조조정 및 M&A 비용",
    "Net Non Operating Interest Income Expense": "영업외 순이자수익/비용",
    "Interest Expense Non Operating": "영업외 이자비용",
    "Interest Income Non Operating": "영업외 이자수익",
    "Operating Income": "영업이익",
    "Operating Expense": "영업비용",
    "Research And Development": "연구개발비",
    "Selling General And Administration": "판매관리비",
    "Gross Profit": "매출총이익",
    "Cost Of Revenue": "매출원가",
    "Total Revenue": "총매출",
    "Operating Revenue": "영업수익"
}

### 1. US데이터 수집

In [6]:
nasdaq_stocks = fdr.StockListing('NASDAQ')
nyse_stocks = fdr.StockListing('NYSE')
all_stocks = pd.concat([nasdaq_stocks, nyse_stocks])

100%|██████████| 2745/2745 [00:02<00:00, 927.48it/s] 


In [ ]:
list_tickers = all_stocks['Symbol'].unique().tolist()

# 중간 저장용 함수
def save_chunk(data, chunk_idx):
    with open(f'pkl/yf_chunk_{chunk_idx}.pkl', 'wb') as f:
        pickle.dump(data, f)

chunk_size = 100
all_data = {}

for chunk_idx in range(0, len(list_tickers), chunk_size):
    chunk = list_tickers[chunk_idx:chunk_idx+chunk_size]
    chunk_data = {}

    for ticker in tqdm(chunk, desc=f'Processing chunk {chunk_idx//chunk_size + 1}'):
        retries = 2
        while retries > 0:
            try:
                tkr = yf.Ticker(ticker)

                # 기본 재무제표, 배당 등
                balance_sheet = tkr.balance_sheet.T
                financials = tkr.financials.T
                dividends = tkr.dividends
                info = tkr.info

                chunk_data[ticker] = {
                    'balance_sheet': balance_sheet,
                    'financials': financials,
                    'dividends': dividends,
                    'info': info
                }

                break
            except Exception as e:
                print(f"{ticker} failed with error: {e}, retries left: {retries-1}")
                retries -= 1
                time.sleep(1)
        time.sleep(0.1)

    # chunk 저장
    save_chunk(chunk_data, chunk_idx//chunk_size + 1)
    all_data.update(chunk_data)

# with open(f'pkl/yf_chunk_all.pkl', 'wb') as f:
#     pickle.dump(all_data, f)

In [7]:
all_data = {}
folder_path = "./pkl"
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
for file_nm in tqdm(files, desc=f'Processing {len(files)} files'):
    dict_file_nm = pd.read_pickle(os.path.join(folder_path, file_nm))
    all_data.update(dict_file_nm)

with open(f'pkl/yf_chunk_all.pkl', 'wb') as f:
    pickle.dump(all_data, f)


Processing 67 files: 100%|██████████| 67/67 [00:04<00:00, 15.77it/s]


### 2. 저가치기반으로 1차 후보 추출

In [28]:
dict_us_stocks = pd.read_pickle('pkl/yf_chunk_all.pkl')
dict_1st_candid_tickers = {'Symbol':[], 'per':[], 'pbr':[], 'eps':[], 'pbs':[], 'peg':[], 'cur_price':[], 'exp_price':[]}
for ticker  in tqdm(dict_us_stocks.keys()):

    if 'info' not in dict_us_stocks[ticker]: continue
    if 'financials' not in dict_us_stocks[ticker]: continue

    info = dict_us_stocks[ticker]['info']
    financials = dict_us_stocks[ticker]['financials']

    eps = info.get('trailingEps')
    pbs = info.get('bookValue')
    per = info.get("trailingPE")
    pbr = info.get("priceToBook")
    cur_price = info.get('currentPrice')

    if per == 'Infinity': continue
    if pd.isna(eps) | pd.isna(pbs) | pd.isna(per) | pd.isna(pbr) | pd.isna(cur_price): continue

    PEG = 100
    if 'Diluted EPS' in financials:
        diluted_eps = [v for v in financials['Diluted EPS'].values if not pd.isna(v) ]
        if len(diluted_eps) < 4: continue
        eps_growth = (np.pow(diluted_eps[0] / diluted_eps[-1], 1/(len(diluted_eps) - 1)) - 1) * 100
        PEG = per / eps_growth if eps_growth > 0 else 100
        exp_price = per * eps_growth if eps_growth > 0 else 100


    if ( (per <= 10) & (pbr <= 1) ) or (PEG <= 1):
        dict_1st_candid_tickers['Symbol'].append(ticker)
        dict_1st_candid_tickers['per'].append(per)
        dict_1st_candid_tickers['pbr'].append(pbr)
        dict_1st_candid_tickers['eps'].append(eps)
        dict_1st_candid_tickers['pbs'].append(pbs)
        dict_1st_candid_tickers['peg'].append(PEG)
        dict_1st_candid_tickers['cur_price'].append(cur_price)
        dict_1st_candid_tickers['exp_price'].append(exp_price)

df_1st_candid = pd.DataFrame(dict_1st_candid_tickers)

  0%|          | 0/6517 [00:00<?, ?it/s]/var/folders/jk/1j1mgc7x11122bdp64mnjcf40000gp/T/ipykernel_32074/1928362550.py:24: RuntimeWarning: invalid value encountered in power
  eps_growth = (np.pow(diluted_eps[0] / diluted_eps[-1], 1/(len(diluted_eps) - 1)) - 1) * 100
/var/folders/jk/1j1mgc7x11122bdp64mnjcf40000gp/T/ipykernel_32074/1928362550.py:24: RuntimeWarning: divide by zero encountered in scalar divide
  eps_growth = (np.pow(diluted_eps[0] / diluted_eps[-1], 1/(len(diluted_eps) - 1)) - 1) * 100
100%|██████████| 6517/6517 [00:00<00:00, 12314.10it/s]


### 3. 1차 후보 도메황인 현황 및 지표 생성

In [24]:
df_first_stocks = all_stocks[all_stocks['Symbol'].isin(df_1st_candid['Symbol'].unique().tolist())]
df_first_stocks.groupby('Industry').size().reset_index(name='count_corp').sort_values(by='count_corp', ascending=False).head(10)

,Industry,count_corp
65,은행,94
107,폐쇄형 펀드,62
48,소프트웨어,26
49,손해보험,23
103,투자 관리 및 펀드 운영,22
62,온라인 서비스,21
84,제약,19
0,IT 서비스 및 컨설팅,18
43,생명 공학 및 의학 연구,18
7,건설 및 엔지니어링,17


In [18]:
dict_symbol_info = {
        'Symbol':[], '시가총액':[], '발행주식수':[], 'EPS(주당순이익)':[], 'BPS(주당순자산)':[],
        'PER':[], 'PBR':[], 'ROE':[], "현재배당수익률":[], "연간배당금":[], "주가":[],
        'count_유동_over_50':[], 'count_당좌_over_30':[], 'count_부채_over_200':[],
        'is_총매출_growth':[], 'is_매출총이익률_growth':[], 'is_영업이익률_growth':[], 'is_당기순이익률_growth':[],
        '평균총매출':[], '평균매출총이익률':[], '평균영업이익률':[], '평균당기순이익률':[], 'is_배당_4':[], 'is_배당_증가':[]
}

for ticker in tqdm(df_first_stocks.Symbol.unique().tolist()):
    info = dict_us_stocks[ticker]['info']
    balance_sheet = dict_us_stocks[ticker]['balance_sheet']
    financials = dict_us_stocks[ticker]['financials']
    dividends = dict_us_stocks[ticker]['dividends']

    balance_sheet.columns = [FIN_COL_MAP_KR_BS[col] if col in FIN_COL_MAP_KR_BS else col for col in balance_sheet.columns]
    list_bs_col = [col for col in balance_sheet.columns if col in ['유동자산', '재고자산', '유동부채', '매출채권', '총부채(비지배제외)', '총자본/주주지분(비지배제외)']]
    df_bs = balance_sheet[list_bs_col].copy()

    financials.columns = [ FIN_COL_MAP_KR_YF[col] if col in FIN_COL_MAP_KR_YF else col for col in financials.columns ]
    list_fin_col = [col for col in financials.columns if col in ['총매출', '매출총이익', '영업이익', '순이익']]
    df_fin = financials[list_fin_col].copy()

    if df_bs.empty or len(df_bs) < 3: continue
    if df_fin.empty or len(df_fin) < 3: continue
    if dividends.empty: continue

    df_base = df_bs.join(df_fin)
    df_base.dropna(inplace=True, how='all')

    count_유동비율_over_50, count_당좌비율_over_30, count_부채비율_over_200 = -1, -1, -1
    if ('유동자산' in df_base.columns) & ('재고자산' in df_base.columns):
        df_base.loc[:, '당좌자산'] = df_base['유동자산'] - df_base['재고자산']
        df_base.loc[:, '유동비율'] = np.round(df_base['유동자산'] / df_base['유동부채'] * 100, 2)
        df_base.loc[:, '당좌비율'] = np.round(df_base['당좌자산'] / df_base['유동부채'] * 100, 2)
        count_유동비율_over_50 = (df_base['유동비율'] > 50).sum()
        count_당좌비율_over_30 = (df_base['당좌비율'] > 30).sum()
    df_base.loc[:, '부채비율'] = np.round(df_base['총부채(비지배제외)'] / df_base['총자본/주주지분(비지배제외)'] * 100, 2)
    count_부채비율_over_200 = (df_base['부채비율'] > 200).sum()

    is_총매출_growth, is_매출총이익률_growth, is_영업이익률_growth, is_당기순이익률_growth = -1, -1, -1, -1
    avg_총매출, avg_매출총이익률, avg_영업이익률, avg_당기순이익률 = -1, -1, -1, -1
    if '총매출' in df_base.columns:
        df_base['총매출_next'] = df_base['총매출'].shift(1)
        df_base.loc[:, '매출성장률'] = np.round((df_base['총매출_next'] - df_base['총매출']) / df_base['총매출'] * 100, 2)
        df_base.loc[:, '매출총이익률'] = np.round(df_base['매출총이익'] / df_base['총매출'] * 100, 2) if '매출총이익' in df_base.columns else -1
        df_base.loc[:, '영업이익률'] = np.round(df_base['영업이익'] / df_base['총매출'] * 100, 2) if '영업이익' in df_base.columns else -1
        df_base.loc[:, '당기순이익률'] = np.round(df_base['순이익'] / df_base['총매출'] * 100, 2) if '순이익' in df_base.columns else -1
        is_총매출_growth = 1 if (df_base['총매출'] < df_base['총매출_next']).all() == True else 0
        is_매출총이익률_growth = 1 if (df_base['매출총이익률'] < df_base['매출총이익률'].shift(1))[1:].all() == True else 0
        is_영업이익률_growth = 1 if (df_base['영업이익률'] < df_base['영업이익률'].shift(1))[1:].all() == True else 0
        is_당기순이익률_growth = 1 if (df_base['당기순이익률'] < df_base['당기순이익률'].shift(1))[1:].all() == True else 0
        avg_총매출 = np.mean(df_base['총매출'])
        avg_매출총이익률 = np.mean(df_base['매출총이익률'])
        avg_영업이익률 = np.mean(df_base['영업이익률'])
        avg_당기순이익률 = np.mean(df_base['당기순이익률'])

    df_dividend = dividends.to_frame(name='dividend').reset_index()
    df_dividend['year'] = df_dividend['Date'].dt.year
    count_dividend_per_year = df_dividend[df_dividend['year'] == 2024].groupby('year')['dividend'].count()
    is_dividend_4 = (count_dividend_per_year.mean() >= 4)
    is_dividend_inc = (df_dividend['dividend'] >= df_dividend['dividend'].shift(1))[1:].all()

    dict_symbol_info["시가총액"].append(info.get("marketCap"))
    dict_symbol_info["발행주식수"].append(info.get("sharesOutstanding"))
    dict_symbol_info["EPS(주당순이익)"].append(info.get("trailingEps"))
    dict_symbol_info["BPS(주당순자산)"].append(info.get("bookValue"))
    dict_symbol_info["PER"].append(info.get("trailingPE"))
    dict_symbol_info["PBR"].append(info.get("priceToBook"))
    dict_symbol_info["ROE"].append(info.get("returnOnEquity"))
    dict_symbol_info["현재배당수익률"].append(info.get('dividendYield'))
    dict_symbol_info["연간배당금"].append(info.get('dividendRate'))
    dict_symbol_info["주가"].append(info.get('currentPrice'))

    dict_symbol_info['Symbol'].append(ticker)
    dict_symbol_info['count_유동_over_50'].append(count_유동비율_over_50)
    dict_symbol_info['count_당좌_over_30'].append(count_당좌비율_over_30)
    dict_symbol_info['count_부채_over_200'].append(count_부채비율_over_200)
    dict_symbol_info['is_총매출_growth'].append(is_총매출_growth)
    dict_symbol_info['is_매출총이익률_growth'].append(is_매출총이익률_growth)
    dict_symbol_info['is_영업이익률_growth'].append(is_영업이익률_growth)
    dict_symbol_info['is_당기순이익률_growth'].append(is_당기순이익률_growth)
    dict_symbol_info['평균총매출'].append(avg_총매출)
    dict_symbol_info['평균매출총이익률'].append(avg_매출총이익률)
    dict_symbol_info['평균영업이익률'].append(avg_영업이익률)
    dict_symbol_info['평균당기순이익률'].append(avg_당기순이익률)
    dict_symbol_info['is_배당_4'].append(is_dividend_4)
    dict_symbol_info['is_배당_증가'].append(is_dividend_inc)


100%|██████████| 800/800 [00:01<00:00, 648.98it/s]


In [31]:
df_indicator = pd.DataFrame(dict_symbol_info)



df_indicator_filtered = df_indicator[(df_indicator['count_유동_over_50'] < 1) & (df_indicator['count_당좌_over_30'] < 1) &
                                     (df_indicator['count_부채_over_200'] < 1) & (df_indicator['is_배당_4'] == True) & (df_indicator['is_배당_증가'] == True)]

df_indicator_filtered = df_indicator_filtered.merge(df_1st_candid, on='Symbol')
df_indicator_filtered.sort_values(by=['per', 'pbr', 'exp_price'], ascending=[True, True, False])

,Symbol,시가총액,발행주식수,EPS(주당순이익),BPS(주당순자산),PER,PBR,ROE,현재배당수익률,연간배당금,주가,count_유동_over_50,count_당좌_over_30,count_부채_over_200,is_총매출_growth,is_매출총이익률_growth,is_영업이익률_growth,is_당기순이익률_growth,평균총매출,평균매출총이익률,평균영업이익률,평균당기순이익률,is_배당_4,is_배당_증가,per,pbr,eps,pbs,peg,cur_price,exp_price
4,LBRDP,NaN,NaN,5.45,61.816,4.485321,0.395448,0.12940,7.20,1.75,24.445,-1,-1,0,0,0,1,0,9.900000e+08,-1.0000,4.7675,89.6675,True,True,4.485321,0.395448,5.45,61.816,100.000000,24.445,224.040348
7,VICI,3.072833e+10,1.068811e+09,2.63,25.892,10.931559,1.110382,0.10357,6.27,1.80,28.750,-1,-1,0,0,1,0,0,2.892864e+09,99.0750,85.7875,62.3275,True,True,10.931559,1.110382,2.63,25.892,0.821720,28.750,145.425448
3,GBLI,3.936222e+08,1.053034e+07,2.01,48.879,13.671641,0.562205,0.04012,5.28,1.40,27.480,-1,-1,0,0,0,0,0,5.690295e+08,-1.0000,-1.0000,4.7000,True,True,13.671641,0.562205,2.01,48.879,0.825405,27.480,226.450996
2,ICFI,1.445930e+09,1.843593e+07,5.41,55.550,14.497228,1.411881,0.09882,0.73,0.56,78.430,-1,-1,0,0,0,0,0,1.829009e+09,36.3225,7.0500,4.4625,True,True,14.497228,1.411881,5.41,55.550,0.901030,78.430,233.254729
9,MMS,4.882132e+09,5.446151e+07,5.44,30.545,15.926471,2.836471,0.18143,1.37,1.20,86.640,-1,-1,0,0,1,0,0,5.068305e+09,22.3500,7.9375,4.8375,True,True,15.926471,2.836471,5.44,30.545,0.849185,86.640,298.700948
1,MORN,9.007056e+09,4.112920e+07,8.93,36.476,23.913773,5.854534,0.24418,0.87,1.82,213.550,-1,-1,0,0,0,0,0,1.970900e+09,59.1225,13.4750,9.5825,True,True,23.913773,5.854534,8.93,36.476,0.977524,213.550,585.017438
0,SBAC,2.060587e+10,1.065630e+08,7.96,-46.210,24.292461,-4.184549,NaN,2.30,4.44,193.368,-1,-1,0,0,0,1,1,2.583376e+09,76.6700,43.5025,18.5725,True,True,24.292461,-4.184549,7.96,-46.210,0.505901,193.368,1166.481193
8,AMH,1.355748e+10,3.704708e+08,1.17,19.297,27.427351,1.662953,0.06605,3.70,1.20,32.090,-1,-1,0,0,1,1,1,1.536680e+09,55.5425,22.7300,20.0250,True,True,27.427351,1.662953,1.17,19.297,0.719760,32.090,1045.153202
5,CRM,2.238187e+11,9.520000e+08,7.09,64.218,33.021156,3.645707,0.11202,0.73,1.66,234.120,-1,-1,0,0,0,1,0,3.264900e+10,74.8775,11.3600,8.5825,True,True,33.021156,3.645707,7.09,64.218,0.527669,234.120,2066.439395
6,HLT,6.696672e+10,2.324352e+08,7.15,-21.163,39.822380,-13.454143,NaN,0.22,0.60,284.730,-1,-1,0,0,0,0,0,8.992500e+09,28.8450,21.1800,11.5700,True,True,39.822380,-13.454143,7.15,-21.163,0.648436,284.730,2445.611761


In [1]:
# dict_symbol_info = {    'Symbol':[], '시가총액':[], '발행주식수':[], 'EPS(주당순이익)':[], 'BPS(주당순자산)':[],
#                         'PER':[], 'PBR':[], 'ROE':[], "현재배당수익률":[], "연간배당금":[], "주가":[],
#                         'count_유동_over_50':[], 'count_당좌_over_30':[], 'count_부채_over_200':[],
#                         'is_총매출_growth':[], 'is_매출총이익률_growth':[], 'is_영업이익률_growth':[], 'is_당기순이익률_growth':[],
#                         '평균총매출':[], '평균매출총이익률':[], '평균영업이익률':[], '평균당기순이익률':[], 'is_배당_4':[], 'is_배당_증가':[]   }
#
#
# list_tickers = all_stocks['Symbol'].unique().tolist()
# for ticker in tqdm(list_tickers):
#
#     tkr = yf.Ticker(ticker)
#     time.sleep(1)
#
#     balance_sheet = tkr.balance_sheet.T
#     balance_sheet.columns = [FIN_COL_MAP_KR_BS[col] if col in FIN_COL_MAP_KR_BS else col for col in balance_sheet.columns]
#     list_bs_col = [col for col in balance_sheet.columns if col in ['유동자산', '재고자산', '유동부채', '매출채권', '총부채(비지배제외)', '총자본/주주지분(비지배제외)']]
#     df_bs = balance_sheet[list_bs_col].copy()
#
#     fin_sheet = tkr.financials.T
#     fin_sheet.columns = [ FIN_COL_MAP_KR_YF[col] if col in FIN_COL_MAP_KR_YF else col for col in fin_sheet.columns ]
#     list_fin_col = [col for col in fin_sheet.columns if col in ['총매출', '매출총이익', '영업이익', '순이익']]
#     df_fin = fin_sheet[list_fin_col].copy()
#
#     dividend_sheet = tkr.dividends
#     info = tkr.info
#
#     if df_bs.empty or len(df_bs) < 3: continue
#     if df_fin.empty or len(df_fin) < 3: continue
#     if dividend_sheet.empty: continue
#
#     df_base = df_bs.join(df_fin)
#     df_base.dropna(inplace=True, how='all')
#
#     #df_base.loc[df_base['재고자산'].isna(), '재고자산'] = 0
#     df_base.loc[:, '당좌자산'] = df_base['유동자산'] - df_base['재고자산'] if '재고자산' in df_base.columns else df_base['유동자산'] if '유동자산' in df_base.columns else -1
#     df_base.loc[:, '유동비율'] = np.round(df_base['유동자산'] / df_base['유동부채'] * 100, 2) if '유동부채' in df_base.columns else -1
#     df_base.loc[:, '당좌비율'] = np.round(df_base['당좌자산'] / df_base['유동부채'] * 100, 2) if '유동부채' in df_base.columns else -1
#     df_base.loc[:, '부채비율'] = np.round(df_base['총부채(비지배제외)'] / df_base['총자본/주주지분(비지배제외)'] * 100, 2)
#     count_유동비율_over_50 = (df_base['유동비율'] < 50).sum()
#     count_당좌비율_over_30 = (df_base['당좌비율'] < 30).sum()
#     count_부채비율_over_200 = (df_base['부채비율'] > 200).sum()
#
#     df_base['총매출_next'] = df_base['총매출'].shift(1)
#     df_base.loc[:, '매출성장률'] = np.round((df_base['총매출_next']  - df_base['총매출'])/ df_base['총매출'] * 100, 2)
#     df_base.loc[:, '매출총이익률'] = np.round(df_base['매출총이익'] / df_base['총매출'] * 100, 2) if '매출총이익' in df_base.columns else -1
#     df_base.loc[:, '영업이익률'] = np.round(df_base['영업이익'] / df_base['총매출'] * 100, 2) if '영업이익' in df_base.columns else -1
#     df_base.loc[:, '당기순이익률'] = np.round(df_base['순이익'] / df_base['총매출'] * 100, 2)
#     df_base.loc[:, '자기자본이익률'] = np.round(df_base['순이익'] / df_base['총자본/주주지분(비지배제외)'] * 100, 2)
#     is_총매출_growth = (df_base['총매출'] < df_base['총매출_next']).all()
#     is_매출총이익률_growth = (df_base['매출총이익률'] < df_base['매출총이익률'].shift(1))[1:].all() if '매출총이익률' in df_base.columns else False
#     is_영업이익률_growth = (df_base['영업이익률'] < df_base['영업이익률'].shift(1))[1:].all() if '영업이익률' in df_base.columns else False
#     is_당기순이익률_growth = (df_base['당기순이익률'] < df_base['당기순이익률'].shift(1))[1:].all()
#     avg_총매출 = np.mean(df_base['총매출'])
#     avg_매출총이익률 = np.mean(df_base['매출총이익률']) if '매출총이익률' in df_base.columns else -1
#     avg_영업이익률 = np.mean(df_base['영업이익률']) if '영업이익률' in df_base.columns else -1
#     avg_당기순이익률 = np.mean(df_base['당기순이익률'])
#
#     df_dividend = dividend_sheet.to_frame(name='dividend').reset_index()
#     df_dividend['year'] = df_dividend['Date'].dt.year
#     count_dividend_per_year = df_dividend[df_dividend['year'] >= 2020].groupby('year')['dividend'].count()
#     is_dividend_4 = (count_dividend_per_year.mean() >= 4)
#     is_dividend_inc = (df_dividend['dividend'] >= df_dividend['dividend'].shift(1))[1:].all()
#
#     dict_symbol_info["시가총액"].append(info.get("marketCap"))
#     dict_symbol_info["발행주식수"].append(info.get("sharesOutstanding"))
#     dict_symbol_info["EPS(주당순이익)"].append(info.get("trailingEps"))
#     dict_symbol_info["BPS(주당순자산)"].append(info.get("bookValue"))
#     dict_symbol_info["PER"].append(info.get("trailingPE"))
#     dict_symbol_info["PBR"].append(info.get("priceToBook"))
#     dict_symbol_info["ROE"].append(info.get("returnOnEquity"))
#     dict_symbol_info["현재배당수익률"].append(info.get('dividendYield'))
#     dict_symbol_info["연간배당금"].append(info.get('dividendRate'))
#     dict_symbol_info["주가"].append(info.get('currentPrice'))
#
#     dict_symbol_info['Symbol'].append(ticker)
#     dict_symbol_info['count_유동_over_50'].append(count_유동비율_over_50)
#     dict_symbol_info['count_당좌_over_30'].append(count_당좌비율_over_30)
#     dict_symbol_info['count_부채_over_200'].append(count_부채비율_over_200)
#     dict_symbol_info['is_총매출_growth'].append(is_총매출_growth)
#     dict_symbol_info['is_매출총이익률_growth'].append(is_매출총이익률_growth)
#     dict_symbol_info['is_영업이익률_growth'].append(is_영업이익률_growth)
#     dict_symbol_info['is_당기순이익률_growth'].append(is_당기순이익률_growth)
#     dict_symbol_info['평균총매출'].append(avg_총매출)
#     dict_symbol_info['평균매출총이익률'].append(avg_매출총이익률)
#     dict_symbol_info['평균영업이익률'].append(avg_영업이익률)
#     dict_symbol_info['평균당기순이익률'].append(avg_당기순이익률)
#     dict_symbol_info['is_배당_4'].append(is_dividend_4)
#     dict_symbol_info['is_배당_증가'].append(is_dividend_inc)
#
#
# df_indicator = pd.DataFrame(dict_symbol_info)
# df_indicator.to_pickle('us_indicator.pkl')